In [1]:
import numpy as np
import pandas as pd
from WordleFunctions import wordle, hardModeFilter, cypher, entropy, nextWord

In [2]:
five = pd.concat([
    pd.read_csv("wordle-answers-alphabetical.txt",header=None,names=["word"]),
    pd.read_csv("wordle-allowed-guesses.txt",header=None,names=["word"])],axis=0,ignore_index=True
)
answers = pd.read_csv("wordle-answers-alphabetical.txt",header=None,names=["word"])

In [3]:
five.head(10)

,word
0,aback
1,abase
2,abate
3,abbey
4,abbot
5,abhor
6,abide
7,abled
8,abode
9,abort


In [4]:
# use wordle() to compute the Wordle response for a given answer and guess
answer="ready"
guess="rhyme"
wordle(answer,guess)

'GBYBY'

In [5]:
# apply a guess to every word in the list
key="ready"
print("Using the guess:",key)
pd.concat([five.word,cypher(five.word,key)],axis=1).head(10)

Using the guess: ready


,word,cypher
0,aback,BBGBB
1,abase,BYGBB
2,abate,BYGBB
3,abbey,BYYBG
4,abbot,BBYBB
5,abhor,YBYBB
6,abide,BYYGB
7,abled,BYYYB
8,abode,BYYGB
9,abort,YBYBB


In [6]:
# compute the entropy of the category responses, considered as a probability distribution for a given guess
guess="tares"
entropy(cypher(five.word,guess))

4.2933900573740065

In [7]:
# prompt user to respond with the Wordle response, e.g. as "BYYBG"
def getResponse():
    while(True):
        print("Enter response: ")
        string=input()
        string=string.upper()
        if len(string)==0:
            return ""
        if len(string)!=5:
            print("Response must be exactly 5 characters")
        else:
            done=1
            for char in string:
                if char not in ("B","G","Y"):
                    print("Enter Wordle response using 'B','Y','G'")
                    done=0
                    break
            if done==1:
                break
    return string

In [8]:
# Recursively find the best word until solved
def solver(dfPossibleAnswers,hardMode):
    guess="soare"
    dfRemainingGuesses = five
    while(True):
        print("Next guess:")
        print("\t",guess)
        sig=getResponse()
        if sig=="":
            break
        if sig=="GGGGG":
            print("Then that's the word!")
            break
        dfPossibleAnswers=dfPossibleAnswers[cypher(dfPossibleAnswers.word,guess)==sig]
        if hardMode:
            dfRemainingGuesses = dfRemainingGuesses[dfRemainingGuesses.word.apply(lambda x: hardModeFilter(x,guess,sig))]
        if dfPossibleAnswers.shape[0]==1:
            print("Then the word must be:")
            print("\t",dfPossibleAnswers.word.iloc[0])
            break
        if dfPossibleAnswers.shape[0]==0:
            print("No such word was found.")
            break
        guess=nextWord(dfPossibleAnswers.word,dfRemainingGuesses.word,hardMode)

In [9]:
solver(answers,True)

Next guess:
	 soare
Enter response: 
yybbg
Next guess:
	 chose
Enter response: 
bgggg
Next guess:
	 those
Enter response: 
ggggg
Then that's the word!
